In [3]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols

def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")

def extract_data(pieces,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 0 
  for files in new_files:
     each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     index = index + len(files)
  print(" ")
  return now

def make_df(now):
   path = f'/content/extracted/{now}/*.npy'
   files = glob.glob(path)
   data = []
   for adr in files:
     data.append(np.load(adr))
   data = np.concatenate(data)
   print(data.shape)
   np.save(f'/content/extracted/{now}/data',data)
   return data
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(np.load(adr))
      datas = np.concatenate(datas)
      rnow = datetime.now().strftime("%H%M%S%f")
      np.save(f"/content/extracted/{now}/part_{rnow}",datas)   
    for adr in addresses:
      os.remove(adr)
def scaler(row,min_range,max_range):
    scaler = MinMaxScaler(feature_range=(min_range, max_range))
    row = scaler.fit_transform(row)
    return row

def each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        #print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 10 == 0:
           print(f"{files.index(file)+1+index}",end=" ")
           if (files.index(file)+index+1) % 400 == 0:
             print(" ")

        address = f"/content/data/{file}"
        #try:
        unattached_dfs.append(process(pd.read_csv(address),min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that))
        #except:
        #  pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     right_now = datetime.now().strftime("%H%M%S%f")
     np.save(f"/content/extracted/{now}/{right_now}",data)  
def process(data,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(each_row_past, data.shape[0]-10):
        row_del = []          
        odel = []
        hdel = []
        ldel = []
        cdel = []
        for ix in range(each_row_past, 1,-1):
            odel.append([data[i-ix][0] - data[i-ix+1][0]])
            hdel.append([data[i-ix][1] - data[i-ix+1][1]])
            ldel.append([data[i-ix][2] - data[i-ix+1][2]])
            cdel.append([data[i-ix][3] - data[i-ix+1][3]])

        
        row_del = np.array([odel,hdel,ldel,cdel])
        
        scaled_row_del = []
        for m in row_del:
          m = np.array(m)
          sc = scaler(m,min_range,max_range)
          scaled_row_del.append(sc)


        row.append(np.array(scaled_row_del).reshape(4,-1))
    return np.array(row)
def start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet):
     folder_name = extract_data(into_how_many_csv,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     to_par(folder_name,csv_to_parquet)
     data = make_df(folder_name)
     return data

In [17]:

min_range,max_range = -1,1
how_many_future_candles = 5
how_many_past_candles = 5
each_row_past,interval_for_that = 50,-1

into_how_many_csv = 20
csv_to_parquet = 1
data = start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet)
#print(yTrain.value_counts())
#print(yTest.value_counts())

Files In Data : 60
Processing File:
10 20 30 40 50 60  
(6845, 4, 49)
(6845, 4, 49)


In [5]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols[:60],"max","1wk")

Symbols : 1500
Data Folder Removed
 


In [19]:
lol = []
data = np.array(data)
for i in range(each_row_past, data.shape[0]-10):
   er = []
   for ix in range(each_row_past, 0,-1):
       er.append(data[i-ix])
   lol.append(er)

In [20]:
np.array(lol).shape

(6785, 50, 4, 49)

In [ ]:
pd.DataFrame(lol[0])

,0,1,2,3,4,5
0,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0
5,399.100006,406.915985,397.130005,402.152008,402.152008,24127600.0
6,402.092010,441.557007,396.196991,435.790985,435.790985,45099500.0
7,435.751007,436.112000,421.131989,423.204987,423.204987,30627700.0
8,423.156006,423.519989,409.467987,411.574005,411.574005,26814400.0
9,411.428986,414.937988,400.009003,404.424988,404.424988,21460800.0


In [ ]:
lol = np.array(lol)

In [ ]:
lol.shape

(2789, 50, 6)

In [22]:
data.shape

(6845, 4, 49)

In [23]:
xTrain = data[:6500]

In [24]:
xTrain.shape

(6500, 4, 49)

In [28]:
xTest = data[6500:]

In [29]:
xTest.shape

(345, 4, 49)

In [30]:
yTrain = []
for i in xTrain:
  if i[3][48] > i[3][43]:
    yTrain.append([1,0])
  else:
    yTrain.append([0,1])


In [31]:
yTrain = np.array(yTrain)

In [32]:
yTrain.shape

(6500, 2)

In [33]:
yTest = []
for i in xTest:
  if i[3][48] > i[3][43]:
    yTest.append([1,0])
  else:
    yTest.append([0,1])


In [34]:
yTest = np.array(yTest)
yTest.shape

(345, 2)

In [35]:
yTrain.shape

(6500, 2)

In [51]:
model = Sequential()

model.add(Dense(500, activation='relu', input_shape=(4,49)))
model.add(Dense(500, activation='relu'))
model.add(Dense(822, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Flatten())
model.add(Dense(350, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 4, 500)            25000     
                                                                 
 dense_40 (Dense)            (None, 4, 500)            250500    
                                                                 
 dense_41 (Dense)            (None, 4, 822)            411822    
                                                                 
 dense_42 (Dense)            (None, 4, 500)            411500    
                                                                 
 flatten_8 (Flatten)         (None, 2000)              0         
                                                                 
 dense_43 (Dense)            (None, 350)               700350    
                                                                 
 dense_44 (Dense)            (None, 300)              

In [ ]:
model.fit(xTrain,yTrain,epochs=15,batch_size=32,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"AL6W_{acr}.h5")

Epoch 1/15
204/204 [==============================] - 9s 39ms/step - loss: 0.3314 - accuracy: 0.8303 - val_loss: 0.2386 - val_accuracy: 0.9101
Epoch 2/15
204/204 [==============================] - 8s 37ms/step - loss: 0.1344 - accuracy: 0.9452 - val_loss: 0.2007 - val_accuracy: 0.9101
Epoch 3/15
204/204 [==============================] - 8s 38ms/step - loss: 0.0991 - accuracy: 0.9569 - val_loss: 0.2296 - val_accuracy: 0.9072
Epoch 4/15
204/204 [==============================] - 8s 38ms/step - loss: 0.0746 - accuracy: 0.9697 - val_loss: 0.1880 - val_accuracy: 0.9217
Epoch 5/15
204/204 [==============================] - 8s 38ms/step - loss: 0.0519 - accuracy: 0.9783 - val_loss: 0.5337 - val_accuracy: 0.8551
Epoch 6/15
204/204 [==============================] - 8s 38ms/step - loss: 0.0452 - accuracy: 0.9835 - val_loss: 0.1525 - val_accuracy: 0.9391
Epoch 7/15
204/204 [==============================] - 8s 38ms/step - loss: 0.0319 - accuracy: 0.9880 - val_loss: 0.1569 - val_accuracy: 0.9391

In [ ]:
def process_for_prediction(data,index,min_range,max_range,each_row_past,interval_for_that):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)

        opens = []
        highs = []
        lows = []
        closes = []
        for ix in range(each_row_past, 1, -1):
            opens.append([data[i-ix][0] - data[i-ix+1][0] ])
            highs.append([data[i-ix][1] - data[i-ix+1][1] ])
            lows.append([data[i-ix][2] - data[i-ix+1][2] ])
            closes.append([data[i-ix][3] - data[i-ix+1][3] ])
        
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        arr = scaler(arr.reshape(-1, 1),min_range,max_range)
        mean_of_previous = []
        for ix in range(i-5,i,):
              mean_of_previous.append((data[ix][3]+data[ix][0])/2)
        mean_of_previous = sum(mean_of_previous)/5
        return arr,mean_of_previous
def make_prediction_for_yf(symbol,period,timeframe,index,min_range,max_range,each_row_past,interval_for_that):
    raw_data, mean_of_previous = process_for_prediction(
        yf.download(symbol, period=period, interval=timeframe), index,min_range,max_range,each_row_past,interval_for_that)
    return f"""YF   : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction_for_tv(symbol,exchange,timeframe,tindex,min_range,max_range,each_row_past,interval_for_that):
    tv = TvDatafeed()
    raw_data, mean_of_previous = process_for_prediction(
       tv.get_hist(symbol=symbol, exchange=exchange, interval=timeframe, n_bars=5000), tindex,min_range,max_range,each_row_past,interval_for_that)
    return f"""TVB  : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index,min_range,max_range,each_row_past,interval_for_that))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex,min_range,max_range,each_row_past,interval_for_that))

In [ ]:
make_prediction("btc-usd","max","1d","btcusd","bitstamp",Interval.in_daily,-1,-1)

In [ ]:
yf.download("btc-usd",period="max",interval="1wk")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')